In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble,metrics
from sklearn import datasets
import lightgbm as lgb

## Data Preparation

In [2]:
data=pd.read_csv('NIFTY.txt',header=None)
data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
toData=data[['Open','Close','Low','Volume']]

def preProcessDataScore(toData):
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

def preProcessDataTrain(toData):
    import pandas as pd
    data=pd.read_csv('../ZMOD/Data/NIFTY.txt',header=None)
    data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
    toData=data[['Open','Close','Low','Volume']]
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

In [3]:
toData=preProcessDataScore(toData)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [4]:
toData.head()
toData.columns

Index(['Open', 'Close', 'Low', 'Volume', 'RollingOpen', 'RollingClose',
       'RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow'],
      dtype='object')

## Model LGB

In [5]:
XVar=['Open', 'Close', 'Low',  'RollingOpen', 'RollingClose','RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow']
YVar=['Volume']

train_data=lgb.Dataset(toData[XVar],toData[YVar])
param = {'num_leaves': 31, 'objective': 'regression','metric':'mean_absolute_error','num_round':50}
model1_ = lgb.train(param, train_data)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [6]:
# model1=lgb.sklearn.LGBMRegressor()

In [7]:
# model1.fit(toData[XVar],toData[YVar])

In [8]:
model1_.predict(toData[XVar])

array([33.15615078, 33.15615078, 33.15615078, ..., 21.30851679,
       21.30851679, 21.30851679])

In [9]:
model1_.best_iteration

0

## PostProcess for 2nd Model

In [10]:
scored=model1_.predict(toData[XVar])

In [11]:
def preProcessData2nd(toData,scored):
    toData['resi']=toData['Volume']-scored
    toData['Rollingresi']=toData.rolling(window=5)[['resi']].mean()
    toData['StdDevresi']=toData.rolling(window=5)[['resi']].std()
    toData=toData.fillna(method='bfill')
    toData['Volume2']=toData['Volume'].apply(lambda x: 1 if x>90 else 0)
    return toData

In [12]:
toData2=preProcessData2nd(toData,scored)

In [13]:
toData2.shape

(92630, 14)

In [14]:
toData2.columns

Index(['Open', 'Close', 'Low', 'Volume', 'RollingOpen', 'RollingClose',
       'RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow', 'resi',
       'Rollingresi', 'StdDevresi', 'Volume2'],
      dtype='object')

In [15]:
toData2.shape

(92630, 14)

In [16]:
XVar2=['Open', 'Close', 'Low',  'RollingOpen', 'RollingClose','RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow', 'resi',
       'Rollingresi', 'StdDevresi']
YVar2=['Volume2']

In [17]:
toData2.head()

,Open,Close,Low,Volume,RollingOpen,RollingClose,RollingLow,StdDevOpen,StdDevClose,StdDevLow,resi,Rollingresi,StdDevresi,Volume2
0,10531.70,10531.70,10531.70,100,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,66.843849,33.043849,18.992104,1
1,10533.75,10535.50,10518.00,57,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,23.843849,33.043849,18.992104,0
2,10523.00,10527.50,10522.25,59,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,25.843849,33.043849,18.992104,0
3,10525.80,10526.15,10522.60,60,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,26.843849,33.043849,18.992104,0
4,10522.30,10522.95,10519.65,55,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,21.843849,33.043849,18.992104,0


## 2nd Model LSTM

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [19]:
kk=toData2[XVar2].shape[0]
hh=toData2[XVar2].shape[1]
XInde=toData2[XVar2].values.reshape(kk,1,hh)
YDepe=toData2[YVar2].values#.reshape(1,kk)

In [20]:
XInde[0].shape

(1, 12)

In [21]:
# model = Sequential()
# model.add(LSTM(4, input_shape=(None, 12)))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(XInde,YDepe ,epochs=100, batch_size=1000, verbose=2)

In [22]:
verbose, epochs, batch_size = 1, 2, 32
#n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(32, input_shape=(None,12)))
# model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XInde,YDepe, epochs=epochs, batch_size=32, verbose=verbose)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
92630/92630 [==============================] - 6s 62us/step - loss: 0.0158 - acc: 0.9986
Epoch 2/2
92630/92630 [==============================] - 5s 55us/step - loss: 0.0082 - acc: 0.9989


In [23]:
from nyoka.keras.keras_model_to_pmml import KerasToPmml

In [24]:
pp=KerasToPmml(model,targetVarName=YVar2[0]).DeepNetwork[0].MiningSchema

Came here


In [25]:
import sys
pp.export(sys.stdout,0)

<MiningSchema>
    <MiningField name="dataSet" usageType="active" invalidValueTreatment="asIs"/>
    <MiningField name="predictions" usageType="target" invalidValueTreatment="asIs"/>
</MiningSchema>


## Score a sample data

In [ ]:
dataTest=pd.read_csv('NIFTY.txt',header=None)
dataTest.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
dataTest=data[['Open','Close','Low','Volume']].head(10)

In [ ]:
dataTest=preProcessDataScore(dataTest)

In [ ]:
scoredTest=model1_.predict(dataTest[XVar])
dataTest2=preProcessData2nd(dataTest,scoredTest)

In [ ]:
dataTest2.shape

In [ ]:
kk2=dataTest2[XVar2].shape[0]
hh2=dataTest2[XVar2].shape[1]
XInde2=dataTest2[XVar2].values.reshape(kk2,1,hh2)
YDepe2=dataTest2[YVar2].values#.reshape(1,kk)

In [ ]:
model.predict(XInde2).tolist()

# PostProcess for 2nd Model

In [ ]:
def postProcessing2(data,scored):
        import requests
        from requests.auth import HTTPBasicAuth
        import json
        import operator

        url = "https://ai.cumulocity.com/alarm/alarms"
        
        for i in range(scored):
            if lst[i] >= .6:
                defect_timestamp = data['timestamp'].values[i]

        headers = {
        'Content-Type': "application/vnd.com.nsn.cumulocity.alarm+json",
        'Accept': "application/vnd.com.nsn.cumulocity.alarm+json",
        }

        payload_Critical_Drink = {
            "source": {
                "id": "117701" },
            "type": "Anamoly",
            "text": "There has been an Anamoly and it occured in this timestamp:"+ str(defect_timestamp),
            "severity": "CRITICAL",
            "status": "ACTIVE",
            "time": "2014-03-03T12:03:27.845Z"
        }


        print ('File read')

        auth = HTTPBasicAuth('Rainer.Burkhardt@softwareag.com', 'cum 2418 Point Loma')

        if pred == 1:
            response = requests.request("POST", url, data=json.dumps(payload_Critical_Drink),headers=headers,auth=auth)

        print ('Done')

# PMML Conversion

In [ ]:
from nyoka import model_to_pmml

In [ ]:
toExportDict={
    'model1':{
        'hyperparameters':param,
        'preProcessingScript':{'scripts':[preProcessDataTrain,preProcessDataScore], 'scriptpurpose':['train','score']},
        'pipelineObj':None,
        'modelObj':model1_,
        'featuresUsed':XVar,
        'targetName':'Volume',
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    },
    'model2':{
    'hyperparameters':None,
    'preProcessingScript':{'scripts':[preProcessData2nd], 'scriptpurpose':['score']},
    'pipelineObj':None,
    'modelObj':model,
    'featuresUsed':XVar2,
    'targetName':'Volume2',
    'postProcessingScript':{'scripts':[postProcessing2], 'scriptpurpose':['score']},
    'taskType': 'score'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="UseCase22Sample.pmml")